In [1]:
import numpy as np

In [ ]:
sentence = "The quick brown fox jumps over the lazy dog"

## Simple tokenization by splitting on spaces, ideally more complex tokenization would be used like BPE or WordPiece
sentence = sentence.split()
n = len(sentence)

print(f"Tokenized sentence: {sentence}")
print(f"Number of tokens: {len(sentence)}")


Tokenized sentence: ['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
Number of tokens: 9


In [ ]:
## Sample word embeddings, ideally these would be learned in the language modelling process or loaded from a pre-trained model like GloVe or Word2Vec

embeddings = np.array([
        [1.0, 0.5, 0.2, 0.8], 
        [0.3, 1.0, 0.7, 0.1],  
        [0.6, 0.2, 1.0, 0.4],  
        [0.9, 0.8, 0.3, 1.0],  
        [0.4, 0.6, 0.8, 0.2],  
        [0.7, 0.3, 0.5, 0.9],  
        [1.0, 0.5, 0.2, 0.8],  
        [0.2, 0.9, 0.4, 0.6],  
        [0.8, 0.4, 0.9, 0.3]  
    ])

print("Word embeddings (4-dimensional):")
for i, word in enumerate(sentence):
    print(f"  {word:4}: {embeddings[i]}")
print()

Word embeddings (4-dimensional):
  The : [1.  0.5 0.2 0.8]
  quick: [0.3 1.  0.7 0.1]
  brown: [0.6 0.2 1.  0.4]
  fox : [0.9 0.8 0.3 1. ]
  jumps: [0.4 0.6 0.8 0.2]
  over: [0.7 0.3 0.5 0.9]
  the : [1.  0.5 0.2 0.8]
  lazy: [0.2 0.9 0.4 0.6]
  dog : [0.8 0.4 0.9 0.3]



In [ ]:
## Sample positional encodings, typically these would be generated using math functions or learned during training or RoPE

positional_embeddings = np.array([
    [0.0, 1.0, 0.0, 1.0],  
    [0.1, 0.9, 0.1, 0.9],  
    [0.2, 0.8, 0.2, 0.8],  
    [0.3, 0.7, 0.3, 0.7],  
    [0.4, 0.6, 0.4, 0.6],  
    [0.5, 0.5, 0.5, 0.5],  
    [0.6, 0.4, 0.6, 0.4],  
    [0.7, 0.3, 0.7, 0.3],  
    [0.8, 0.2, 0.8, 0.2]   
])

print("Positional embeddings (Same dimesnions as word embeddings):")
for i, word in enumerate(sentence):
    print(f"  Pos {i} ({word}): {positional_embeddings[i]}")
print()

Positional embeddings (Same dimesnions as word embeddings):
  Pos 0 (The): [0. 1. 0. 1.]
  Pos 1 (quick): [0.1 0.9 0.1 0.9]
  Pos 2 (brown): [0.2 0.8 0.2 0.8]
  Pos 3 (fox): [0.3 0.7 0.3 0.7]
  Pos 4 (jumps): [0.4 0.6 0.4 0.6]
  Pos 5 (over): [0.5 0.5 0.5 0.5]
  Pos 6 (the): [0.6 0.4 0.6 0.4]
  Pos 7 (lazy): [0.7 0.3 0.7 0.3]
  Pos 8 (dog): [0.8 0.2 0.8 0.2]



In [7]:
## The final input to the Self Attention block is the sum of the word embeddings and positional encodings

input_embeddings = embeddings + positional_embeddings

print("Input embeddings (word + positional):")
for i, word in enumerate(sentence):
    print(f"  {word:4}: {input_embeddings[i]}")
print()

Input embeddings (word + positional):
  The : [1.  1.5 0.2 1.8]
  quick: [0.4 1.9 0.8 1. ]
  brown: [0.8 1.  1.2 1.2]
  fox : [1.2 1.5 0.6 1.7]
  jumps: [0.8 1.2 1.2 0.8]
  over: [1.2 0.8 1.  1.4]
  the : [1.6 0.9 0.8 1.2]
  lazy: [0.9 1.2 1.1 0.9]
  dog : [1.6 0.6 1.7 0.5]



In [9]:
d_model = input_embeddings.shape[1]  # Embedding dimension
d_k = 3 # Dimension of keys and queries (generally kept smaller to make Q, K and V matrices low rank for efficiency)

In [11]:
np.random.seed(42)  # For reproducible results
W_q = np.random.randn(d_model, d_k) * 0.5  # Query weights
W_k = np.random.randn(d_model, d_k) * 0.5  # Key weights  
W_v = np.random.randn(d_model, d_k) * 0.5  # Value weights

print("Weight matrices:")
print(f"W_q (Query weights) shape: {W_q.shape}")
print(W_q)
print(f"\nW_k (Key weights) shape: {W_k.shape}")
print(W_k)
print(f"\nW_v (Value weights) shape: {W_v.shape}")
print(W_v)
print()

Weight matrices:
W_q (Query weights) shape: (4, 3)
[[ 0.24835708 -0.06913215  0.32384427]
 [ 0.76151493 -0.11707669 -0.11706848]
 [ 0.78960641  0.38371736 -0.23473719]
 [ 0.27128002 -0.23170885 -0.23286488]]

W_k (Key weights) shape: (4, 3)
[[ 0.12098114 -0.95664012 -0.86245892]
 [-0.28114376 -0.50641556  0.15712367]
 [-0.45401204 -0.70615185  0.73282438]
 [-0.11288815  0.0337641  -0.71237409]]

W_v (Value weights) shape: (4, 3)
[[-0.27219136  0.05546129 -0.57549679]
 [ 0.18784901 -0.30031934 -0.14584687]
 [-0.30085331  0.92613909 -0.00674861]
 [-0.52885546  0.41127246 -0.61042182]]



In [13]:
## Computing Q, K and V for each token

Q = input_embeddings @ W_q  # Shape: (n, d_k)
K = input_embeddings @ W_k  # Shape: (n, d_k)  
V = input_embeddings @ W_v  # Shape: (n, d_k)

print(f"Q (Queries) shape: {Q.shape}")
for i, word in enumerate(sentence):
    print(f"  {word:4}: {Q[i]}")
print(f"\nK (Keys) shape: {K.shape}")
for i, word in enumerate(sentence):
    print(f"  {word:4}: {K[i]}")
print(f"\nV (Values) shape: {V.shape}")
for i, word in enumerate(sentence):
    print(f"  {word:4}: {V[i]}")
print()

Q (Queries) shape: (9, 3)
  The : [ 2.03685479 -0.58507963 -0.31786267]
  quick: [ 2.44918634 -0.17483352 -0.51354703]
  brown: [ 2.2332643   0.01002781 -0.41911555]
  fox : [ 2.37524077 -0.42224823 -0.3237022 ]
  jumps: [ 2.27705528  0.07929602 -0.34938329]
  over: [ 2.07663887 -0.11729495 -0.26578968]
  the : [ 2.03995591 -0.18705718 -0.05443841]
  lazy: [ 2.25005835  0.01084018 -0.31681163]
  dog : [ 2.33225118  0.35560764 -0.06757592]

K (Keys) shape: (9, 3)
  The : [-0.59473559 -1.79671845 -1.76248191]
  quick: [-0.96187848 -1.87600299 -0.17256319]
  brown: [-0.86463908 -2.07859296 -0.50830312]
  fox : [-0.74085536 -2.27388362 -1.57060653]
  jumps: [-0.87571257 -2.19338171 -0.19192875]
  over: [-0.6917931  -2.2119827  -1.17375111]
  the : [-0.55813498 -2.51080276 -1.50711237]
  lazy: [-0.82950207 -2.21505413 -0.42269449]
  dog : [-0.80338098 -3.01804963 -0.39604566]

V (Values) shape: (9, 3)
  The : [-1.00252835  0.53050052 -1.89437611]
  quick: [-0.52150154  0.60376149 -1.1231284

In [ ]:
attention_scores = Q @ K.T  # Shape: (n, n)

## For every token(row), we have a attention score wrt it for every other token (column)
print(f"Attention scores shape: {attention_scores.shape}")
print(attention_scores)

Attention scores shape: (9, 9)
[[ 0.40006053 -0.80674425 -0.38343127  0.32062538 -0.4393894   0.2581956
   0.8112344  -0.25923342  0.2553171 ]
 [-0.23737432 -1.93921211 -1.49321695 -0.61036143 -1.66274219 -0.70482508
  -0.154031   -1.42726593 -1.23658541]
 [-0.60753535 -2.09461717 -1.7387736  -1.0190623  -1.89725214 -1.07520084
  -0.63998657 -1.69755169 -1.65843761]
 [-0.08345976 -1.43669495 -1.01150496 -0.29115773 -1.0917489  -0.32922356
   0.22233266 -0.89813731 -0.50565629]
 [-0.8809367  -2.27871934 -1.95606251 -1.31853485 -2.10091567 -1.34056351
  -0.94344098 -1.91678465 -1.93029048]
 [-0.55585554 -1.73156305 -1.41663295 -0.85432297 -1.51025349 -0.8651791
  -0.4639654  -1.35041375 -1.20906534]
 [-0.78119858 -1.60187579 -1.34733865 -1.00046469 -1.36567894 -0.93356302
  -0.58686227 -1.25479505 -1.05275382]
 [-0.79929176 -2.12994889 -1.80698436 -1.19403068 -1.93337578 -1.20869512
  -0.8055831  -1.75652512 -1.71489841]
 [-1.90689825 -2.89880211 -2.72137001 -2.43033601 -2.80939523 -2.32

In [ ]:
##Scaling the attention scores by sqrt(d_k) for stability to prevent large dot products

scaled_scores = attention_scores / np.sqrt(d_k) # Shape: (n, n)

print(f"Sclaed attention scores shape: {scaled_scores.shape}")
print(scaled_scores)

Sclaed attention scores shape: (9, 9)
[[ 0.23097506 -0.46577401 -0.22137415  0.18511315 -0.25368159  0.1490693
   0.4683664  -0.14966848  0.14740739]
 [-0.13704813 -1.11960464 -0.86210921 -0.35239234 -0.95998465 -0.40693095
  -0.08892984 -0.82403237 -0.71394292]
 [-0.3507607  -1.20932779 -1.00388141 -0.5883559  -1.09537903 -0.6207675
  -0.36949642 -0.98008192 -0.9574994 ]
 [-0.04818551 -0.82947621 -0.58399266 -0.16809999 -0.63032152 -0.19007731
   0.12836382 -0.51853982 -0.29194079]
 [-0.50860904 -1.31561922 -1.12933322 -0.76125645 -1.21296423 -0.7739747
  -0.5446959  -1.10665613 -1.11445373]
 [-0.32092334 -0.99971839 -0.81789341 -0.4932436  -0.87194526 -0.49951139
  -0.26787055 -0.77966174 -0.6980542 ]
 [-0.45102521 -0.92484342 -0.77788633 -0.57761856 -0.7884751  -0.53899286
  -0.33882509 -0.72445626 -0.6078077 ]
 [-0.46147131 -1.22972656 -1.04326291 -0.68937393 -1.11623503 -0.69784045
  -0.46510362 -1.01413025 -0.99009706]
 [-1.10094822 -1.67362418 -1.57118371 -1.40315515 -1.62200509

In [16]:
def softmax(x):
    # Subtract max for numerical stability
    exp_x = np.exp(x - np.max(x))
    # Normalize over rows to get probabilities i.e. sum of each row = 1 and for each token, we have a pdf over all tokens
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

In [ ]:
## For every token(row), we have a attention weight wrt it for every other token (column). The attention weights show how much each word 'attends to' every other word. Higher weights mean stronger relationships/dependencies.
attention_weights = softmax(scaled_scores) # Shape: (n, n)

print(f"Attention weights shape: {attention_weights.shape}")
print(attention_weights)

Attention weights shape: (9, 9)
[[0.13336955 0.06644502 0.08484064 0.12739111 0.08214346 0.12288121
  0.16910422 0.09114762 0.12267716]
 [0.16718085 0.06258463 0.08096485 0.13479188 0.07341583 0.12763739
  0.17542199 0.08410718 0.0938954 ]
 [0.16582324 0.07027073 0.08629759 0.13075525 0.07875203 0.1265852
  0.16274535 0.08837607 0.09039453]
 [0.14371987 0.06579707 0.0841044  0.12747899 0.08029682 0.12470791
  0.17147121 0.08979342 0.1126303 ]
 [0.16442247 0.07336368 0.08838612 0.12771379 0.08129495 0.12609978
  0.15859476 0.09041336 0.08971109]
 [0.14831268 0.07522832 0.09022916 0.1248362  0.08548157 0.1240562
  0.15639355 0.09374557 0.10171674]
 [0.13177547 0.0820461  0.09503434 0.1161063  0.09403336 0.12067872
  0.14742205 0.10025014 0.11265353]
 [0.15900324 0.07374914 0.0888663  0.12659866 0.08261249 0.12553134
  0.15842674 0.09149329 0.0937188 ]
 [0.15625735 0.08813135 0.0976382  0.1155031  0.09280008 0.12304949
  0.14036076 0.09920861 0.08705105]]


In [ ]:
## Since we are using sample weights, the attention weights may not be very interpretable. But we can still see which words attend most strongly to which other words.

for i, word in enumerate(sentence):
        max_attention_idx = np.argmax(attention_weights[i])
        max_attention_word = sentence[max_attention_idx]
        max_attention_score = attention_weights[i, max_attention_idx]
        print(f"'{word}' attends most strongly to '{max_attention_word}' (weight: {max_attention_score:.3f})")

'The' attends most strongly to 'the' (weight: 0.169)
'quick' attends most strongly to 'the' (weight: 0.175)
'brown' attends most strongly to 'The' (weight: 0.166)
'fox' attends most strongly to 'the' (weight: 0.171)
'jumps' attends most strongly to 'The' (weight: 0.164)
'over' attends most strongly to 'the' (weight: 0.156)
'the' attends most strongly to 'the' (weight: 0.147)
'lazy' attends most strongly to 'The' (weight: 0.159)
'dog' attends most strongly to 'The' (weight: 0.156)


In [19]:
output = attention_weights @ V  # Shape: (n, d_k)

print(f"\nOutput shape: {output.shape}")
print(output)


Output shape: (9, 3)
[[-1.01519543  1.07533453 -1.56726705]
 [-1.02021325  1.03911995 -1.5986345 ]
 [-1.01186097  1.03666559 -1.58629144]
 [-1.01591323  1.06393395 -1.57551695]
 [-1.00836894  1.03684566 -1.58026968]
 [-1.00603878  1.0529471  -1.56617407]
 [-0.99780319  1.06833541 -1.54254752]
 [-1.00781032  1.04236163 -1.57580933]
 [-0.99238985  1.03795567 -1.552786  ]]
